In [19]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [20]:
df = pd.read_json("/shared/3/projects/benlitterer/podcastData/processed/floydMonth/floydMonthData.jsonl", orient="records", lines=True)

In [ ]:
#just in case 
inDf = df

In [111]:
df = df.drop_duplicates(subset=["potentialOutPath"])

In [113]:
#do we want to work with podcasts where there's no description? or no transcript? let's say no for now
df = df.dropna(subset=["rssUrl", "transcript", "enclosure", "potentialOutPath", "epDescription"])

In [114]:
df = df[df["transcript"].apply(lambda x: len(x)) > 0]
df = df[df["epDescription"].apply(lambda x: len(x)) > 0]

In [116]:
print(df.shape)

(558301, 69)


In [117]:
#TODO: sample repeat episodes within podcasts
#so maybe sample podcasts first then episodes  
sample = df[["rssUrl", "transcript", "enclosure", "cleanDatesLoc", "epTitle", "title", "podDescription", "epDescription"]].sample(1000, random_state=120)

In [118]:
#TODO: clean out html tags and stuff like that...
#TODO: just replace tags with a space and make spaces and then make one space between every word  
DESC_LEN=100
TRANS_LEN=300

def cleaner(inStr): 
    #clean out all tags in brackets or parenthesis
    inStr = re.sub(r'\[[^()]*\]\s*', '', inStr)
    inStr = re.sub(r'\([^()]*\)\s*', '', inStr)
    
    #replace paragraph breaks with new lines  
    inStr = re.sub("</p>",". ", inStr)
    inStr = re.sub("</br>", ". ", inStr)

    #replace all other tags with space
    #inStr = re.sub("<.+?>", " ", inStr)

    #replace all amounts of space with a single space 
    #inStr = re.sub(r"[ \t]+", " ", inStr)
        
    return inStr

def makePresentationText(inRow): 
    description = inRow["epDescription"]
    transcript = inRow["transcript"]

    description = cleaner(description)
    transcript = cleaner(transcript)

    #description = " ".join(BeautifulSoup(description, "html.parser").text.split())
    description = BeautifulSoup(description, "html.parser").text
    transcript = " ".join(BeautifulSoup(transcript, "html.parser").text.split())

    if len(description.split()) > DESC_LEN: 
        description = " ".join(description.split()[:DESC_LEN]) + "..."
    
    if len(transcript.split()) > TRANS_LEN: 
        transcript = " ".join(transcript.split()[:TRANS_LEN]) + "..."
    
    return f"<strong>Description:</strong><br>{description}<br><br><strong>Transcript:</strong><br>\n{transcript}"

In [119]:
sample.apply(makePresentationText, axis=1)

/tmp/ipykernel_2083121/3861952727.py:31: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, "html.parser").text
/tmp/ipykernel_2083121/3861952727.py:31: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  description = BeautifulSoup(description, "html.parser").text


374181    <strong>Description:</strong><br>This week we'...
414326    <strong>Description:</strong><br>The where and...
520892    <strong>Description:</strong><br>Brother M, Co...
568899    <strong>Description:</strong><br>Working actor...
97683     <strong>Description:</strong><br>Even in these...
                                ...                        
488824    <strong>Description:</strong><br>Food for the ...
471760    <strong>Description:</strong><br>You cannot me...
10703     <strong>Description:</strong><br>Today, we bre...
17138     <strong>Description:</strong><br>God has given...
489686    <strong>Description:</strong><br>We are all he...
Length: 1000, dtype: object

In [120]:
sample["displayText"] = sample.apply(makePresentationText, axis=1)

/tmp/ipykernel_2083121/3861952727.py:31: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, "html.parser").text
/tmp/ipykernel_2083121/3861952727.py:31: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  description = BeautifulSoup(description, "html.parser").text


In [121]:
for dispText in list(sample["displayText"])[:10]: 
    print(dispText)

<strong>Description:</strong><br>This week we're reading a story by A.S. Byatt about a couple of upper-class twits who get their comeuppance. You love to see it! Also, in light of the recent dustup over Curtis Sittenfeld's Rodhman, we talk about alternate-universe novels we'd like to see in the world. Plus a new segment: Dante's Inferno!. If you like the show, and would like more Book Fight in your life, please consider subscribing to our Patreon. For $5, you'll get access to three bonus episodes a month, including Book Fight After Dark, where we read some of the world's weirdest--and steamiest!--novels. We've also...<br><br><strong>Transcript:</strong><br>
*The Antherian -, "The Antherian"" plays* Hello and welcome to Book Fight, a podcast about reading and writing that has been called many different things in the years since it began, but never once painterly. I'm Mike Ingram, I'm here with Tom Calister. Tom, have you ever described something as painterly? I have not, I don't think i

In [122]:
sample.to_json("/shared/3/projects/benlitterer/podcastData/annotation/1000sample.jsonl", orient="records", lines=True)